In [1]:
import torch
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
from torchvision import transforms

In [2]:
data_transform = transforms.Compose([transforms.ToTensor(),
                                     transforms.Normalize((0.5,), (0.5,))])

train_data = MNIST(root="./data", train=True, download=True, transform=data_transform)
test_data = MNIST(root="./data", train=False, download=True, transform=data_transform)

In [3]:
print("Training data length: ", len(train_data))
print("Test data length: ", len(test_data))

Training data length:  60000
Test data length:  10000


In [4]:
batch_size = 64

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

In [5]:
from model import Classifier

net = Classifier()
print(net)
#net.load_model("./mnist_model.pt")

Classifier(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.25, inplace=False)
  (fc1): Linear(in_features=800, out_features=400, bias=True)
  (fc2): Linear(in_features=400, out_features=10, bias=True)
)


In [6]:
def train(num_epochs, model):
    for epoch in range(num_epochs):
        running_loss = 0.0
        for i, data in enumerate(train_loader):
            #unpack
            inputs, labels = data

            #reset the gradients
            optimizer.zero_grad()

            #forward
            outputs = model(inputs)

            #loss
            loss = criterion(outputs, labels)

            #calculate backprop weights
            loss.backward()

            #apply learning
            optimizer.step()

            #add loss
            running_loss += loss.item()

            if (i % 100) == 99:
                mean_loss = running_loss / 100
                print("Epoch ", epoch + 1, "; Batch", i + 1, "; Train loss: ", mean_loss)
                running_loss = 0.0

In [7]:
import torch.optim as optim
import torch.nn as nn

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.005, momentum=0.9)

In [8]:
epochs = 30

train(epochs, net)
print("Finished training!")

Epoch  1 ; Batch 100 ; Train loss:  2.192326328754425
Epoch  1 ; Batch 200 ; Train loss:  1.140281103849411
Epoch  1 ; Batch 300 ; Train loss:  0.5629602247476577
Epoch  1 ; Batch 400 ; Train loss:  0.3253117306530476
Epoch  1 ; Batch 500 ; Train loss:  0.26084064081311226
Epoch  1 ; Batch 600 ; Train loss:  0.22310261286795138
Epoch  1 ; Batch 700 ; Train loss:  0.20458401769399642
Epoch  1 ; Batch 800 ; Train loss:  0.18667182497680188
Epoch  1 ; Batch 900 ; Train loss:  0.17509812459349633
Epoch  2 ; Batch 100 ; Train loss:  0.1574059896916151
Epoch  2 ; Batch 200 ; Train loss:  0.14297232896089554
Epoch  2 ; Batch 300 ; Train loss:  0.13662599951028823
Epoch  2 ; Batch 400 ; Train loss:  0.13932531051337718
Epoch  2 ; Batch 500 ; Train loss:  0.13653416607528926
Epoch  2 ; Batch 600 ; Train loss:  0.12195083867758512
Epoch  2 ; Batch 700 ; Train loss:  0.13043525429442526
Epoch  2 ; Batch 800 ; Train loss:  0.12230964431539178
Epoch  2 ; Batch 900 ; Train loss:  0.126055430714041
E

In [11]:
torch.save(net.state_dict(), "./mnist_model.pt")

In [13]:
## testing the model by hand

net.eval()
image, label = next(iter(test_loader))

import numpy as np

num = np.random.randint(batch_size)
x = torch.unsqueeze(image[num], 0)
print(x.shape)
with torch.no_grad():
    print("Predicted:", torch.argmax(net(x)).item(), ", True:", label[num].item())

torch.Size([1, 1, 28, 28])
Predicted: 5 , True: 5


In [14]:
def test_model(test_dataloader, model):
    correct_pred = 0
    model.eval()

    for images, labels in test_dataloader:
        for i in range(len(labels)):
            image = torch.unsqueeze(images[i], 0)

            with torch.no_grad():
                output = model(image)

            label_predicted = torch.argmax(output).item()
            label_true = labels[i].item()

            if label_predicted == label_true:
                correct_pred += 1

    total = len(test_data)

    return correct_pred / total


In [15]:
accuracy = test_model(test_loader, net)

print(f"The accuracy of the model on the test dataset: {accuracy * 100}%")

The accuracy of the model on the test dataset: 99.35000000000001%
